# Fine-tuning with Low-Rank Adaptation (LoRA)

In [ ]:
!pip install wandb python-dotenv datasets peft

In [7]:
import wandb
from dotenv import load_dotenv

from peft import LoraConfig, TaskType, get_peft_model
import transformers
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

transformers.logging.set_verbosity_error()

In [8]:
load_dotenv()
wandb.login()

True

## Dataset loading

In [52]:
dataset = (load_dataset('csv', data_files={'train': '../data/train.csv', 'dev': '../data/dev.csv', 'test': '../data/test.csv'})
           .remove_columns(['id', 'humor_rating', 'humor_controversy', 'offense_rating', 'sentence_length'])
           .rename_column('is_humor', 'label'))

## Model setup

In [53]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [63]:
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/6400 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [55]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

## LoRA Setup 

In [56]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=1,
    lora_alpha=1,
    lora_dropout=0.1,
    bias="none"
)

In [57]:
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

trainable params: 38,402 || all params: 109,522,180 || trainable%: 0.035063217331868304


In [58]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # accuracy = accuracy_score(labels, predictions)
    # precision = precision_score(labels, predictions, average='weighted')
    # recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        # 'accuracy': accuracy,
        # 'precision': precision,
        # 'recall': recall,
        'f1': f1
    }
    

In [64]:
dataset['train']

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6400
})

## Training setup

In [65]:
training_arguments = TrainingArguments(
    output_dir='bert-lora-humor-detection',
    evaluation_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    report_to="wandb",
    run_name="bert-lora-humor-01"
)

In [66]:
trainer = Trainer(
    model=peft_model,
    args=training_arguments,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [67]:
%%time
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 6.18 GB, other allocations: 11.69 GB, max allowed: 18.13 GB). Tried to allocate 384.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [26]:
trainer.predict(dataset['test']).metrics

{'test_loss': 0.3577665686607361,
 'test_f1': 0.8537736197348548,
 'test_runtime': 37.9098,
 'test_samples_per_second': 21.103,
 'test_steps_per_second': 2.638}